In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

In [2]:
from config import g_key
gmaps.configure(api_key=g_key)

In [3]:
cities_csv = "../weatherpy/results/results.csv"
cities_df = pd.read_csv(cities_csv)
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Keïta,100,NE,1651937041,10,14.7551,5.7749,106.86,7.07
1,Airai,47,TL,1651937042,96,-8.9266,125.4092,60.53,3.94
2,Cape Town,0,ZA,1651936746,57,-33.9258,18.4232,63.12,23.02
3,Olyka,26,UA,1651937042,62,50.7186,25.8125,63.93,1.70
4,Kawalu,100,ID,1651936952,96,-7.3817,108.2082,74.03,1.92


In [4]:
coords = cities_df[["Lat", "Lng"]]
humidity = cities_df['Humidity']
humidity_max = humidity.max()


fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coords, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=humidity_max,
                                 point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
temp_cities = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) & (cities_df["Cloudiness"] == 0), :]
temp_cities = temp_cities.dropna()
temp_cities.reset_index(inplace=True)
del temp_cities['index']
temp_cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Richards Bay,0,ZA,1651936778,86,-28.7830,32.0377,70.70,5.17
1,Tigre,0,AR,1651937075,69,-34.4260,-58.5796,72.25,9.22
2,Saint George,0,US,1651936895,18,37.1041,-113.5841,77.79,8.05
3,La Constancia,0,MX,1651937081,56,24.2231,-107.1869,77.74,3.04
4,Martil,0,MA,1651937087,68,35.6166,-5.2752,71.56,8.05


In [6]:
hotels = []

for i in range(len(temp_cities)):
    lat = temp_cities.loc[i]['Lat']
    lng = temp_cities.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append(np.nan)
temp_cities["Hotel Name"] = hotels
vaca_df = temp_cities.dropna()
vaca_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Richards Bay,0,ZA,1651936778,86,-28.7830,32.0377,70.70,5.17,BON Hotel Waterfront Richards Bay
1,Tigre,0,AR,1651937075,69,-34.4260,-58.5796,72.25,9.22,Puerto Delta Apartamentos
2,Saint George,0,US,1651936895,18,37.1041,-113.5841,77.79,8.05,Best Western Plus Abbey Inn
3,La Constancia,0,MX,1651937081,56,24.2231,-107.1869,77.74,3.04,Mi casa
4,Martil,0,MA,1651937087,68,35.6166,-5.2752,71.56,8.05,فندق مرتيل


In [7]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in vaca_df.iterrows()]
locations = vaca_df[["Lat", "Lng"]]

In [8]:
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))